In [14]:
import numpy as np
import pandas as pd
import json
import requests
import csv
from statistics import median

In [15]:

ballot_df = pd.read_csv('/data/rf5_synthetic_ballots.csv')  # Replace 'data.csv' with your actual file path
df = ballot_df[ballot_df['Status'].eq('SUBMITTED')]

json_data = df.to_json(orient='records')

parsed_json = json.loads(json_data)
pretty_data = json.dumps(parsed_json, indent=4)

data = parsed_json
num_raw_ballots = len(data)

In [16]:
url = "https://optimism.easscan.org/graphql"
headers = {
    'Content-Type': 'application/json'
}

attestations = pd.DataFrame()
schemaID = "0x41513aa7b99bfea09d389c74aacedaeb13c28fb748569e9e2400109cbe284ee5"

# Query to filter by schemaID
attest_data = {
    "query": f"""
        query Attestations {{
            attestations(where: {{
                schemaId: {{ equals: "{schemaID}" }},

            }}) {{
                id
                attester
                recipient
                refUID
                decodedDataJson
            }}
        }}
    """,
    "variables": {}
}

# Send the request using POST and json parameter
response = requests.post(url, headers=headers, json=attest_data)
dataset = response.json()

attest_df = pd.json_normalize(dataset['data']['attestations'])
voting_group = []
round = []

for i in attest_df['decodedDataJson']:
  dataset = json.loads(i)
  voting_group.append(dataset[3]['value']['value'])
  round.append(dataset[1]['value']['value'])

attest_df['voting_group'] = voting_group
attest_df['round'] = round

def group_naming(group):
    match group:
        case 'A':
            return 'ETHEREUM_CORE_CONTRIBUTIONS'
        case 'B':
            return 'OP_STACK_RESEARCH_AND_DEVELOPMENT'
        case 'C':
            return 'OP_STACK_TOOLING'

attest_df['category_assignment'] = attest_df['voting_group'].apply(group_naming)
# attest_df.to_csv("rpgf5_category_assignment.csv", encoding='utf-8') # [FOR AUDIT/TESTING]

In [17]:
### Calculate Budget allocation

budget_allocation =[]

addresses = []
# Collect budgets based on category assignment
for entry in data:

    if entry['Address'].lower() not in attest_df['recipient'].str.lower().unique():
      continue

    addresses.append(entry['Address'])
    entry_payload = json.loads(entry["Payload"])
    budget = entry_payload["budget"]
    budget_allocation.append(budget)

budget_median = np.median(budget_allocation)

# print(budget_median) # [FOR AUDIT/TESTING]

In [18]:
### Calculate Category allocation

# 1. Isolate the category budget votes: Each badgeholder will have voted on how to allocate OP to all categories (e.g. [Category1: 33%; Category2: 33%; Category3: 34%])
# 2. Calculate the median of Category allocation
# 3. Adjust category allocations to match 100%

category_scores = {
    'ETHEREUM_CORE_CONTRIBUTIONS': [],
    'OP_STACK_RESEARCH_AND_DEVELOPMENT': [],
    'OP_STACK_TOOLING': []
}

for entry in data:

    if entry['Address'].lower() not in attest_df['recipient'].str.lower().unique():
      print(entry['Address'])
      continue

    entry_payload = json.loads(entry["Payload"])
    allocations = entry_payload["category_allocations"]
    for item in allocations:
      for category, score in item.items():
          category_scores[category].append(float(score))

# Calculate the median for each category
category_medians = {category: np.median(scores) for category, scores in category_scores.items()}

# Normalize the category medians
total_median = sum(category_medians.values())
normalized_category_medians = {category: (median / total_median) * 100 for category, median in category_medians.items()}

# Output the normalized category allocations [FOR AUDIT/TESTING]
# for category, score in normalized_category_medians.items():
#     print(f"{category}: {score}")

In [19]:
### Calculate Project Scores

# 1. Isolate the Project votes: Each badgeholder will have voted by submitting percentages reflecting the allocation of OP to the projects within a category (e.g. [Project1: 10; Project2: 4;….]).
# 2. Remove Project votes with a COI: For each badgeholder, they will have a number of `null` votes. These should not be considered for results calculation
# 3. Calculate the median scores of projects
# 4. Adjust scores to match 100%
# 5. Repeat step 1 - 4 for all 3 categories

# Calculate median project allocation under each category
project_allocations_by_category = {
    'ETHEREUM_CORE_CONTRIBUTIONS': {},
    'OP_STACK_RESEARCH_AND_DEVELOPMENT': {},
    'OP_STACK_TOOLING': {}
}

accepted_entries = []

# Collect project allocations based on category assignment
for entry in data:
    filtered_df = attest_df.loc[((attest_df['recipient'].str.lower() == entry['Address'].lower()) & (attest_df['round'] == str(5)))]
    entry_update = json.loads(entry["Payload"])

    if not filtered_df.empty:

      entry_update = json.loads(entry["Payload"])

      category = filtered_df.iloc[0]["category_assignment"]
      project_allocations = entry_update["project_allocations"]
      accepted_entries.append(entry['Address'])

      for item in project_allocations:
        for project_id, allocation in item.items():
            if allocation is not None:  # Exclude None values
                if project_id not in project_allocations_by_category[category]:
                    project_allocations_by_category[category][project_id] = []
                project_allocations_by_category[category][project_id].append(allocation)
    else:
      print(entry['Address'])

# Calculate median project allocation for each project under each category
median_project_allocations = {}
for category, projects in project_allocations_by_category.items():
    median_project_allocations[category] = {}

    for project_id, allocations in projects.items():
        median_project_allocations[category][project_id] = median(map(float, allocations))

normalized_project_scores = {}

for category, scores in median_project_allocations.items():
    total_score = sum(scores.values())
    normalized_project_scores[category] = {project_id: (score / total_score) * 100 for project_id, score in scores.items()}


# Output the normalized project scores [FOR TESTING]
# for category, scores in normalized_project_scores.items():
#     print(f"\n{category}:")
#     print(f"no. of prjects: {len(scores)}")
#     for project_id, normalized_score in scores.items():
#         print(f"Project ID: {project_id}, Normalized Score: {normalized_score:.2f}%")

# print(f"\n\nAccepted total of {len(accepted_entries)} ballots (out of {num_raw_ballots} submitted).")

In [20]:

### Calculate Results (weighted proportional distribution)

# 1. Multiply Project Scores with category scores - done
# 2. Adjust scores to match 100% (there should be no need to readjust) - done
# 3. Implement Max scores and redistribute excess
# 4. Implement min and redistribute excess (while not breaking max rule)

# Constants
total_amount = budget_median
max_amount = total_amount*0.125
min_amount = 1000

# Total allocation for each category
category_total_allocations = {category: (allocation / 100) * total_amount for category, allocation in normalized_category_medians.items()}
final_allocations = {}

# Sort projects within each category
for category, scores in normalized_project_scores.items():
    normalized_project_scores[category] = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

# Calculate project allocations using weighted proportional distribution
for category, scores in normalized_project_scores.items():
    total_allocation = category_total_allocations[category]
    project_allocations = {}

    # Total score for the category
    total_score = sum(scores.values())

    # Weighted proportional allocations
    for project, score in scores.items():
        allocation = (score / total_score) * total_allocation
        project_allocations[project] = allocation

    # Remove projects below min_amount
    project_allocations = {project: allocation for project, allocation in project_allocations.items() if allocation >= min_amount}

    # Normalize allocations
    total_allocated = sum(project_allocations.values())

    if total_allocated > 0:
        normalized_allocations = {project: (allocation / total_allocated) * total_allocation for project, allocation in project_allocations.items()}
    else:
        normalized_allocations = {}

    # Cap allocations at max_amount
    normalized_allocations = {project: min(allocation, max_amount) for project, allocation in normalized_allocations.items()}

    # Store the final allocations for the category
    final_allocations[category] = normalized_allocations

# Display the final allocations [FOR TESTING]
# for category, allocations in final_allocations.items():
#     print(f"Allocations for {category}:")
#     for project, allocation in allocations.items():
#         print(f"  Project {project}: {allocation:,.2f}")


In [21]:
# Write results into CSV file
csv_file_name = 'rpgf5_allocations_final_result.csv'


with open(csv_file_name, mode='w', newline='') as csv_file:

    writer = csv.writer(csv_file)
    writer.writerow(['Category', 'Project', 'Allocation'])
    for category, allocations in final_allocations.items():
        for project, allocation in allocations.items():
            writer.writerow([category, project, f"{allocation:,.2f}"])

print(f"Allocations have been written to {csv_file_name}")

Allocations have been written to rpgf5_allocations_final_result.csv
